In [1]:
import sys
sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/Modules')
sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/Viz')
sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/MCPC_Clustering/Codes')

datapath='/Users/chandrasekharmukherjee/Home/Centrality framework/SCRNA_benchmark/'


In [2]:
#Import packages
#imports
import time
import pandas as pd
from collections import Counter
import sknetwork


from sknetwork.ranking import PageRank
from sknetwork.ranking import Betweenness
from sknetwork.ranking import Closeness

import umap as umap

from numba.typed import List
import warnings
from numba import njit
import pynndescent
import numpy as np
from sklearn.cluster import SpectralClustering
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
import operator
from sklearn.utils.extmath import randomized_svd
from random import randint
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
import scipy
from umap.umap_ import *
import math
from random import randint
import keras
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.datasets import cifar100
from keras.datasets import fashion_mnist
import scanpy
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_mutual_info_score, roc_auc_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

import community as community_louvain
from sklearn.cluster import KMeans
from sklearn.metrics import fowlkes_mallows_score

from plots import *
import scipy

import igraph 
import networkx as nx


from sklearn.metrics.cluster import normalized_mutual_info_score as NMI


In [5]:
%load_ext autoreload
%autoreload 2
import importlib

from plots import *


import benchmark as bmarks
import metric as met 
import FlowRank as algo
import datasets as dsets
import simulation as simm
import embedding as embed
import newflow as newalgo
import Relative_centrality_v2 as new_score
import Real_Custom_v2 as MCPC_cluster

met=importlib.reload(met)
newalgo=importlib.reload(newalgo)
algo = importlib.reload(algo)
dsets = importlib.reload(dsets)
simm = importlib.reload(simm)
embed=importlib.reload(embed)
bmarks=importlib.reload(bmarks)
new_score=importlib.reload(new_score)


MCPC_cluster=importlib.reload(MCPC_cluster)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
datanames = ['Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe','Xin', 'Zhengmix8eq','Tcell-medicine','ALM', 'AMB', 'TM', 'VISP']


In [19]:
name='Zhengmix8eq'
kchoice=20

X = scipy.sparse.load_npz(datapath+name + '/data.npz')
label = np.load(datapath+name+'/labels.npy')
print(name,len(label))

#Log transform+PCA
X.data = np.log1p(X.data)
print("Log transform done")
pca = TruncatedSVD(n_components=50)
PX = pca.fit_transform(X)
n=PX.shape[0]
walk_len_c1=int(np.log2(n))
print(PX.shape)


#Calculte inital KNN accuracy
met.KNN_graph_acc(PX,kchoice,0,label)

#Get the KNN edgelist
edge_list,vlist=embed.dir_KNN_graph(PX,kchoice,0)
print(len(edge_list))

Zhengmix8eq 3994
Log transform done
(3994, 50)
Accuracy of  20 -NN graph is 0.784
79880


In [8]:
# This code is to test Louvain Method with custom Modularity Function
import numpy as np
import networkx as nx
import networkx.algorithms.community.quality as nx_quality
import networkx.algorithms.community as nx_comm
# from Custom_Func import directed_modularity, modularity, _one_level

import Custom_Func as Custom_Func

In [9]:
from collections import defaultdict, deque

def get_labels(partition,n_s):
    final_partition_1 = deque(partition, maxlen=1).pop()
    #print(final_partition_1)


    label_1=np.zeros((n_s))
    c=0
    for sets in final_partition_1:
        for ell in sets:
            label_1[ell]=c
        
        c=c+1

    return label_1

In [23]:
G = nx.DiGraph()
for (u,v) in edge_list:
    G.add_edge(u,v,weight=1)

#nx_comm.louvain._one_level=Custom_Func._one_level

res_list=[0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.5,2,2.5,3,4]

for res in res_list:
    partition = nx_comm.louvain.louvain_partitions(G, seed=0,resolution=res)
    label_new=get_labels(partition,n)
    print(res,round(NMI(label,label_new),2),round(met.purity_score(label,label_new),2))

print("\n\n")

res_list=[1,2,3,4,5,6,7,8,9,10,11,12]


for res in res_list:
    partition1=MCPC_cluster.louvain_partitions(G, seed=0,resolution=res)
    label_new1=get_labels(partition1,n)
    print(res,round(NMI(label,label_new1),2),round(met.purity_score(label,label_new1),2))


0.5 0.8 0.67
0.6 0.76 0.74
0.7 0.77 0.74
0.8 0.74 0.81
0.9 0.73 0.81
1 0.72 0.8
1.1 0.71 0.78
1.2 0.7 0.81
1.3 0.69 0.81
1.5 0.7 0.81
2 0.68 0.81
2.5 0.68 0.82
3 0.66 0.82
4 0.63 0.83



1 0.8 0.66
2 0.8 0.66
3 0.77 0.67
4 0.74 0.73
5 0.72 0.73
6 0.7 0.73
7 0.73 0.8
8 0.71 0.81
9 0.71 0.82
10 0.7 0.8
11 0.71 0.82
12 0.7 0.83


In [21]:
print(len(set(label_new1)))

5
